In [1]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_video_L.mp4.pkl'
orientation_R = '../calibration/orientation_video_R.mp4.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)
AAAAAAAAAAAAAAAAAAAAAAAAAAa
[array([[ 1.30658287],
       [-0.24183669],
       [-0.36029694]]), array([[0.8370347 ],
       [0.50877275],
       [1.42079534]])]
rvec:
[[ 0.91981346  0.12222957 -0.37283123]
 [-0.39134766  0.21773042 -0.89411435]
 [-0.02811051  0.96832505  0.24810563]]
tvec:
[array([ 392.85668783,  778.93718914, -413.99540707])]
[[ 392.85668783]
 [ 778.93718914]
 [-413.99540707]]
rt matrix:
[[ 9.19813463e-01  1.22229568e-01 -3.72831231e-01  3.92856688e+02]
 [-3.91347663e-01  2.17730418e-01 -8.94114351e-01  7.78937189e+02]
 [-2.81105116e-02  9.68325048e-01  2.48105625e-01 -4.13995407e+02]]

rvec:
[[ 0.11724762 -0.64837912  0.75223502]
 [ 0.97850234 -0.05398333 -0.19904515]
 [ 0.16966487  0.75940129  0.62811106]]
tvec:
[array([-531.92933052,  299.82680741, -762.3439487 ])]
[[-531.92933052]
 [ 299.82680741]
 [-762.3439487 ]]
rt matrix:
[[ 1.17247615e-01 -6.48379116e-01  7.52235016e-01 -5.31929331e+02]
 [ 9.78502340e-01 

In [3]:

zero_point_L = (834, 751)
zero_point_R = (767, 887)

triangulator.triangulate([zero_point_L, zero_point_R])
print(triangulator.projection_matrices[0] @ [0, 0, 0, 1])
print(triangulator.extrinsics[0] @ [0, 0, 0, 1])

[ 1.38878569e+05  8.34472582e+05 -4.13995407e+02]
[ 392.85668783  778.93718914 -413.99540707    1.        ]


In [45]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

In [47]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()